Read the files 2017_german_election_overall.csv and 2017_german_election_party.csv from the german-election-2017 dataset.

In [25]:
import pandas as pd
import numpy as np

For each area, compute the percentage of the voters over the registered voters

In [26]:
german_party = pd.read_csv('database/german-election-2017/2017_german_election_party.csv')
german_overall = pd.read_csv('database/german-election-2017/2017_german_election_overall.csv')

For each state, compute the total number of registered voters and of total votes

In [27]:
german_overall.groupby('state').sum()[['valid_first_votes', 'registered.voters']]

,valid_first_votes,registered.voters
state,,
Baden-Württemberg,5981553,7732570
Bayern,7370973,9519914
Berlin,1864501,2503053
Brandenburg,1488332,2051507
Bremen,330114,474097
Hamburg,974503,1296624
Hessen,3344456,4409241
Mecklenburg-Vorpommern,926877,1323999
Niedersachsen,4634003,6124125


For each state, compute the number of votes for each party (considering only the first votes)

In [28]:
german_party.groupby(['state', 'party']).sum()['votes_first_vote']

state              party                                                                                        
Baden-Württemberg  Ab.jetzt...Demokratie.durch.Volksabstimmung                                                            0
                   Allianz.Deutscher.Demokraten                                                                           0
                   Allianz.für.Menschenrechte.Tier..und.Naturschutz                                                       0
                   Alternative.für.Deutschland                                                                       689893
                   Bayernpartei                                                                                           0
                   BÜNDNIS.90.DIE.GRÜNEN                                                                             801603
                   Bündnis.C...Christen.für.Deutschland                                                                 407
                   

For each state and each party, compute the area where the party has taken the most and the least total votes (considering only the first votes)

First we compute, for each state and each party, the maximum number of votes over all areas

In [29]:
most_votes = german_party.groupby(['state', 'party'])['votes_first_vote'].max()
most_votes.head()

state              party                                           
Baden-Württemberg  Ab.jetzt...Demokratie.durch.Volksabstimmung             0
                   Allianz.Deutscher.Demokraten                            0
                   Allianz.für.Menschenrechte.Tier..und.Naturschutz        0
                   Alternative.für.Deutschland                         28574
                   Bayernpartei                                            0
Name: votes_first_vote, dtype: int64

Then we extract the rows of `german_party` with the specified `state`, `party` and number of votes. This requires to use the [iteritems](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iteritems.html) methods that allows to iterate over all rows of a DataFrame.
The desired results will be stored in the dictionary `best_areas`.

In [30]:
best_areas = {}
for row in most_votes.iteritems():
    data = german_party[(german_party['state'] == row[0][0]) &
             (german_party['party'] == row[0][1]) &
             (german_party['votes_first_vote'] == row[1])].head(1)
    best_areas[(row[0][0], row[0][1])] = data['area_name'].iloc[0]
best_areas

{('Baden-Württemberg',
  'Ab.jetzt...Demokratie.durch.Volksabstimmung'): 'Stuttgart I',
 ('Baden-Württemberg', 'Allianz.Deutscher.Demokraten'): 'Stuttgart I',
 ('Baden-Württemberg',
  'Allianz.für.Menschenrechte.Tier..und.Naturschutz'): 'Stuttgart I',
 ('Baden-Württemberg', 'Alternative.für.Deutschland'): 'Heilbronn',
 ('Baden-Württemberg', 'Bayernpartei'): 'Stuttgart I',
 ('Baden-Württemberg', 'BÜNDNIS.90.DIE.GRÜNEN'): 'Stuttgart I',
 ('Baden-Württemberg', 'Bündnis.C...Christen.für.Deutschland'): 'Freiburg',
 ('Baden-Württemberg', 'Bündnis.Grundeinkommen'): 'Stuttgart I',
 ('Baden-Württemberg', 'Bürgerrechtsbewegung.Solidarität'): 'Stuttgart II',
 ('Baden-Württemberg',
  'Christlich.Demokratische.Union.Deutschlands'): 'Aalen – Heidenheim',
 ('Baden-Württemberg',
  'Christlich.Soziale.Union.in.Bayern.e.V.'): 'Stuttgart I',
 ('Baden-Württemberg', 'DEMOKRATIE.IN.BEWEGUNG'): 'Stuttgart I',
 ('Baden-Württemberg', 'DIE.EINHEIT'): 'Stuttgart I',
 ('Baden-Württemberg', 'DIE.LINKE'): 'Tübingen

To compute the area with the least votes, we use the `argmin` approach.
The main limitation of this approach is that `argmin` is not an `agg` function, that is it cannot be applied directly to each group (the `sum` function is an example of `agg` function).

Therefore we have to extract the rows of each group with a `groupby`, and use `apply` which specificies a function to be applied to each group: this function (in this case) will be the extraction of the index corresponding to the minimum value.

In [31]:
least_index = german_party.groupby(['state', 'party']).apply(lambda group:group['votes_first_vote'].argmin())
german_party.loc[least_index][['state', 'party', 'area_name', 'votes_first_vote']]

,state,party,area_name,votes_first_vote
4443,Baden-Württemberg,Ab.jetzt...Demokratie.durch.Volksabstimmung,Stuttgart I,0
6237,Baden-Württemberg,Allianz.Deutscher.Demokraten,Stuttgart I,0
6536,Baden-Württemberg,Allianz.für.Menschenrechte.Tier..und.Naturschutz,Stuttgart I,0
2051,Baden-Württemberg,Alternative.für.Deutschland,Stuttgart I,10646
4144,Baden-Württemberg,Bayernpartei,Stuttgart I,0
1172,Baden-Württemberg,BÜNDNIS.90.DIE.GRÜNEN,Odenwald – Tauber,11090
10423,Baden-Württemberg,Bündnis.C...Christen.für.Deutschland,Stuttgart I,0
7134,Baden-Württemberg,Bündnis.Grundeinkommen,Stuttgart I,0
5342,Baden-Württemberg,Bürgerrechtsbewegung.Solidarität,Böblingen,0
274,Baden-Württemberg,Christlich.Demokratische.Union.Deutschlands,Mannheim,41809


For each state and each party, compute the area where the party has taken the most and the least votes, as a percentage of the overall registered voters (considering only the first votes).

The first step is to obtain a dictionary `registered` that associates the number of registered voters in each area

In [32]:
registered = german_overall.groupby(['state', 'area_names']).sum()['registered.voters'].to_dict()
registered

{('Baden-Württemberg', 'Aalen – Heidenheim'): 221916,
 ('Baden-Württemberg', 'Backnang – Schwäbisch Gmünd'): 176967,
 ('Baden-Württemberg', 'Biberach'): 167025,
 ('Baden-Württemberg', 'Bodensee'): 172783,
 ('Baden-Württemberg', 'Bruchsal – Schwetzingen'): 196426,
 ('Baden-Württemberg', 'Böblingen'): 245680,
 ('Baden-Württemberg', 'Calw'): 198020,
 ('Baden-Württemberg', 'Emmendingen – Lahr'): 218613,
 ('Baden-Württemberg', 'Esslingen'): 168715,
 ('Baden-Württemberg', 'Freiburg'): 222308,
 ('Baden-Württemberg', 'Göppingen'): 178409,
 ('Baden-Württemberg', 'Heidelberg'): 217934,
 ('Baden-Württemberg', 'Heilbronn'): 242555,
 ('Baden-Württemberg', 'Karlsruhe-Land'): 209741,
 ('Baden-Württemberg', 'Karlsruhe-Stadt'): 208868,
 ('Baden-Württemberg', 'Konstanz'): 205618,
 ('Baden-Württemberg', 'Ludwigsburg'): 218071,
 ('Baden-Württemberg', 'Lörrach – Müllheim'): 231734,
 ('Baden-Württemberg', 'Mannheim'): 197283,
 ('Baden-Württemberg', 'Neckar-Zaber'): 229280,
 ('Baden-Württemberg', 'Nürtingen'

Then we use the dictionary to add the number of registered voters to the `german_party` DataFrame.

In [33]:
german_party['registered_voters'] = german_party.apply(lambda row: registered[(row['state'], row['area_name'])], axis=1)
german_party.head()

,Unnamed: 0,area_id,area_name,state,party,votes_first_vote,votes_second_vote,registered_voters
0,1,1,Flensburg – Schleswig,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,68102,58307,225659
1,2,2,Nordfriesland – Dithmarschen Nord,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,62260,52933,186384
2,3,3,Steinburg – Dithmarschen Süd,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,54812,47367,175950
3,4,4,Rendsburg-Eckernförde,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,66625,56584,199632
4,5,5,Kiel,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,45691,40011,204650


Now we can compute the new column `percentage`

In [34]:
german_party['percentage'] = german_party['votes_first_vote'] / german_party['registered_voters'] * 100
german_party.head()

,Unnamed: 0,area_id,area_name,state,party,votes_first_vote,votes_second_vote,registered_voters,percentage
0,1,1,Flensburg – Schleswig,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,68102,58307,225659,30.179164
1,2,2,Nordfriesland – Dithmarschen Nord,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,62260,52933,186384,33.404155
2,3,3,Steinburg – Dithmarschen Süd,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,54812,47367,175950,31.152032
3,4,4,Rendsburg-Eckernförde,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,66625,56584,199632,33.373908
4,5,5,Kiel,Schleswig-Holstein,Christlich.Demokratische.Union.Deutschlands,45691,40011,204650,22.326411


Now we can use `argmin` and `argmax` to solve the problem

In [35]:
least_perc_index = german_party.groupby(['state', 'party']).apply(lambda group:group['percentage'].argmin())
german_party.loc[least_perc_index][['state', 'party', 'area_name', 'percentage']]

,state,party,area_name,percentage
4443,Baden-Württemberg,Ab.jetzt...Demokratie.durch.Volksabstimmung,Stuttgart I,0.000000
6237,Baden-Württemberg,Allianz.Deutscher.Demokraten,Stuttgart I,0.000000
6536,Baden-Württemberg,Allianz.für.Menschenrechte.Tier..und.Naturschutz,Stuttgart I,0.000000
2051,Baden-Württemberg,Alternative.für.Deutschland,Stuttgart I,5.477239
4144,Baden-Württemberg,Bayernpartei,Stuttgart I,0.000000
1172,Baden-Württemberg,BÜNDNIS.90.DIE.GRÜNEN,Odenwald – Tauber,5.254306
10423,Baden-Württemberg,Bündnis.C...Christen.für.Deutschland,Stuttgart I,0.000000
7134,Baden-Württemberg,Bündnis.Grundeinkommen,Stuttgart I,0.000000
5342,Baden-Württemberg,Bürgerrechtsbewegung.Solidarität,Böblingen,0.000000
274,Baden-Württemberg,Christlich.Demokratische.Union.Deutschlands,Mannheim,21.192399


In [36]:
max_perc_index = german_party.groupby(['state', 'party']).apply(lambda group:group['percentage'].argmax())
german_party.loc[max_perc_index][['state', 'party', 'area_name', 'percentage']]

,state,party,area_name,percentage
4443,Baden-Württemberg,Ab.jetzt...Demokratie.durch.Volksabstimmung,Stuttgart I,0.000000
6237,Baden-Württemberg,Allianz.Deutscher.Demokraten,Stuttgart I,0.000000
6536,Baden-Württemberg,Allianz.für.Menschenrechte.Tier..und.Naturschutz,Stuttgart I,0.000000
2072,Baden-Württemberg,Alternative.für.Deutschland,Pforzheim,11.919281
4144,Baden-Württemberg,Bayernpartei,Stuttgart I,0.000000
1154,Baden-Württemberg,BÜNDNIS.90.DIE.GRÜNEN,Stuttgart I,24.402165
10446,Baden-Württemberg,Bündnis.C...Christen.für.Deutschland,Freiburg,0.183079
7134,Baden-Württemberg,Bündnis.Grundeinkommen,Stuttgart I,0.000000
5341,Baden-Württemberg,Bürgerrechtsbewegung.Solidarität,Stuttgart II,0.061370
283,Baden-Württemberg,Christlich.Demokratische.Union.Deutschlands,Offenburg,35.964311


For each area, compute the difference between the valid first votes and the valid second votes

In [37]:
german_party['difference'] = german_party['votes_first_vote'] - german_party['votes_second_vote']
german_party.groupby('area_name').sum()['difference']

area_name
Aachen I                                                      -445
Aachen II                                                     -443
Aalen – Heidenheim                                            -627
Ahrweiler                                                     -395
Altmark                                                       -410
Altötting                                                     -372
Amberg                                                        -357
Anhalt                                                        -330
Ansbach                                                       -513
Aschaffenburg                                                 -545
Augsburg-Land                                                 -584
Augsburg-Stadt                                                -414
Aurich – Emden                                               -1566
Backnang – Schwäbisch Gmünd                                   -513
Bad Kissingen                                       

For each state, compute the difference between the valid first votes and the valid second votes

In [38]:
german_party.groupby('state').sum()['difference']

state
Baden-Württemberg        -11811
Bayern                   -21451
Berlin                    -3340
Brandenburg               -2427
Bremen                    -1625
Hamburg                   -2767
Hessen                    -4323
Mecklenburg-Vorpommern     -644
Niedersachsen            -11723
Nordrhein-Westfalen      -39164
Rheinland-Pfalz          -10622
Saarland                    123
Sachsen                   -9646
Sachsen-Anhalt            -8551
Schleswig-Holstein        -2650
Thüringen                 -1706
Name: difference, dtype: int64

For each party, compute the difference between the valid first votes and the valid second votes

In [39]:
german_party.groupby('party').sum()['difference']

party
Ab.jetzt...Demokratie.durch.Volksabstimmung                                                        -3579
Allianz.Deutscher.Demokraten                                                                      -41178
Allianz.für.Menschenrechte.Tier..und.Naturschutz                                                  -32218
Alternative.für.Deutschland                                                                      -560999
Bayernpartei                                                                                        4604
BÜNDNIS.90.DIE.GRÜNEN                                                                            -440128
Bündnis.C...Christen.für.Deutschland                                                                1715
Bündnis.Grundeinkommen                                                                            -97386
Bürgerrechtsbewegung.Solidarität                                                                    9258
Christlich.Demokratische.Union.Deutschlands      

For each area and each party, compute the difference between the valid first votes and the valid second votes

In [40]:
german_party.groupby(['area_name', 'party']).sum()['difference']

area_name  party                                                                                        
Aachen I   Ab.jetzt...Demokratie.durch.Volksabstimmung                                                        -67
           Allianz.Deutscher.Demokraten                                                                      -435
           Allianz.für.Menschenrechte.Tier..und.Naturschutz                                                     0
           Alternative.für.Deutschland                                                                       -453
           Bayernpartei                                                                                         0
           BÜNDNIS.90.DIE.GRÜNEN                                                                            -4812
           Bündnis.C...Christen.für.Deutschland                                                                 0
           Bündnis.Grundeinkommen                                                                